# Dolby.io  Media APIs 101 - Getting Started
### Getting Started

**Agenda**

1. The Media API workflow and your Dolby.io account ~ 10 minutes
2. The Enhance API ~ 10 minutes
3. The Diagnose API ~ 5 minutes
4. The Transcode API ~ 10 Minutes
5. The Analyze API ~ 2 minutes
6. The Music Mastering API ~ 2 minutes
7. Webhooks ~ 5 minutes
8. Questions ~ 10 minutes

### The Media API Workflow

**What are media APIs**


- The Dolby.io Media APIs adhere to REpresentational State Transfer or REST architecture style. This means they can be interacted with in an language or framework, be that Kotlin, Python or C. 
- The Media APIs bring programmatic solutions to challenges faced in the Media space...
- - Poor audio quality
- - Converting between audio and video codecs 
- - Mastering music
- - Deriving insight and data from audio
- - And More...

![Workflow](images/workflow.PNG)

### API Keys & Dolby.io

https://dashboard.dolby.io/signin

## What can we use the Dolby.io Media APIs for?

First, let's set our variables.

In [2]:
import IPython

# Enter your Dolby.io Media API Key here.
api_key = "vWsARnDLxteBWgIgOaV7eHW5eHsomkCr"
# Enter your name here to customize the output URL later.
name = "Braden"

print("API Key and Name set!")

API Key and Name set!


## Upload local files

Let's say our files are not currently accessible on a public URL. While you can use your cloud storage providers to [presign URLs temporarily](https://docs.dolby.io/media-apis/docs/media-input-and-output), this doesn't solve the local file issue. In comes the Input APIs.

We will begin with a file already stored locally: `white_noise_demo.wav` as seen in the sidebar.

In [1]:
# Example Local File
IPython.display.Audio("white_noise_demo.wav")

NameError: name 'IPython' is not defined

We can take this file and upload it with another API call!

In [3]:
import requests

# Set or replace these values

file_path = "white_noise_demo.wav"
input_url = f"dlb://in/input-{name}.wav" # Setting the input URL to have a different location based on your name!

url = "https://api.dolby.com/media/input"
headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

# Declare the dlb:// location we want to store the file in
body = {
    "url": input_url,
}

response = requests.post(url, json=body, headers=headers)
response.raise_for_status()
data = response.json()
presigned_url = data["url"]

# Upload your media to the pre-signed url response

print("Uploading {0} to {1}".format(file_path, presigned_url))
with open(file_path, "rb") as input_file:
  requests.put(presigned_url, data=input_file)

HTTPError: 401 Client Error: Unauthorized for url: https://api.dolby.com/media/input

And now the file can be accessed via our new `dlb://` url. This media is stored in an isolated location that only you can access with your API key. It is encrypted at rest and while in transit with https. Access to the production system is controlled and audited to prevent unauthorized use.
> Note: this storage is TEMPORARY and will be removed within 24 - 48 hours.

For Output, also note that you can save the files directly to your cloud provider of choice when done processing, no need to download every complete job! Read more at the documentation for your Cloud Storage Provider of choice here: https://docs.dolby.io/media-apis/docs/media-input-and-output

## Enhance

In part one, we called enhance without using any parameters. We can fine tune Enhance with a few different parameters to optimize for common audio issues including:

- Noise reduction
- Speech leveling and isolation
- Loudness correction
- Content tuning
- Sibilance reduction
- Plosive reduction
- Dynamic equalization
- Tone shaping
- Hum reduction
- Mouth click reduction

**When defining the body**
- Input location
- Output location

Input and output can be a cloud storage provider.

- Content type
    - conference
    - interview
    - lecture
    - meeting
    - mobile_phone
    - music
    - podcast
    - studio
    - voice_over or voice_recording
    
Content type can be left undefined. Enhance will work it out.

- Audio parameters
    - loudness
    - dynamics
    - noise
    - filter
    - speech
    - music
    
Audio parameters can be left undefined, all objects will default to enabled.

Audio parameters have sub-features, for example:
- Speech
    - isolation 
    - sibilance (harsh consonant sound like "s", "sh", "x", "ch", "t", and "th")
    - plosive (pops from "p" and "b")
    - click

See more: https://docs.dolby.io/media-apis/docs/speech

**Lots to choose from depending on your use case**

**Let's see an example**

In [ ]:
#URL changes
url = "https://api.dolby.com/media/enhance"

output_url = f"dlb://out/output-{name}.wav" # Setting the input URL to have a different location based on your name!

#Header stays the same
headers = {
  "x-api-key": api_key,
  "Content-Type": "application/json",
  "Accept": "application/json"
}

body = {
  "input" : input_url,
  "output" : output_url,
  "content" : {
      "type": "voice_over"},
  "audio" : {
    "noise": {
      "reduction": {
        "amount": "max"
      }
    },
    "filter":{
      "hum": {
        "enable": True
      }
    },
    "speech": {
      "isolation": {
          "enable": True,
          "amount": 70
      },
      "sibilance": {
        "reduction": {
              "enable": True,
              "amount": "medium"
        }
      },
      "plosive": {
        "reduction": {
              "enable": True,
              "amount": "medium"
        }
      },
      "click": {
        "reduction": {
              "enable": False,
              "amount": "medium"
        }
      }
    }
  }
}

response = requests.post(url, json=body, headers=headers)
print(response.json())

**Checking the status once again:**

In [ ]:
params = {
  "job_id": response.json()["job_id"]
}

stat = requests.get(url, params=params, headers=headers)
print(stat.json())

**Download the output**

In [ ]:
url = "https://api.dolby.com/media/output"

params = {
    "url": output_url
}

import shutil #Package for file operations
output_path = "white_noise_demo-enhanced.wav"

with requests.get(url, params=params, headers=headers, stream=True) as response:
    response.raw.decode_content = True
    print("Downloading from {0} into {1}".format(response.url, output_path))
    with open(output_path, "wb") as output_file:
        shutil.copyfileobj(response.raw, output_file)

`white_noise_demo-enhanced.wav` will appear in the sidebar when complete.

**Lets give it a listen**

In [ ]:
IPython.display.Audio("white_noise_demo-enhanced.wav")

**Original**

In [ ]:
IPython.display.Audio("white_noise_demo.wav")

### Questions?

https://docs.dolby.io/media-apis/docs/enhance-api-guide

## Diagnose (*Beta*)

- Provides General Media Info
- Audio Quality 
- Noise Score
- Clipping
- Loudness
- Content Classification

Diagnose is **fast**, and doesn't require a file to be downloaded - the results are in the same API call as the status.

Read more about audio quality here: https://docs.dolby.io/media-apis/docs/audio-quality

**Let's look at some code**

In [ ]:
# Note the new URL endpoint
url = "https://api.dolby.com/media/diagnose"

headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

body = {
    "input"  : input_url
}

response = requests.post(url, json=body, headers=headers)
job_id  = response.json()["job_id"]
print(response.json())

**Checking Status and Getting Results**

In [ ]:
url = "https://api.dolby.com/media/diagnose"
headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

params = {
    "job_id" : job_id,
}

stat = requests.get(url, params=params, headers=headers)
IPython.display.JSON(stat.json()) # Display interactive JSON element instead of just printing

You should see a response with collapsible objects like the following:

![diagnose output](./images/diagnose_output.png)

An example workflow of how these APIs would be used in a real world setting might be the following:

![flow](./images/flow.jpg)

### Questions?

https://docs.dolby.io/media-apis/docs/diagnose-api-guide

### A quick note on other languages & our API reference

- cURL Designed to transfer data using various network protocols
- Works great with REST APIs like the Dolby.io Media APIs
- Run through the terminal or PowerShell

**Examples for many languages in the docs:** (Shell, JavaScript, Ruby, PHP, etc.)
- https://docs.dolby.io/media-apis/reference/media-diagnose-post
- https://docs.dolby.io/media-apis/reference/media-diagnose-get

Easiest to do in Postman Collection:
https://www.postman.com/dolbyio/workspace/dolby-io-media-apis/overview

> **Keep your API keys private**

https://docs.dolby.io/media-apis/docs/authentication

Have the option to use OAuth2 Tokens instead if you want it to expire naturally. Rotate your API keys often, as you have an API key for each application on your Dashboard! 

## Transcoding

- Convert media of a particular file type to another
- Transizing video media to different resolutions
- Transrating audio media to different sample rates or bit rates

The goal of Transcoding is to convert from one media format to another, from one media format to many variations, or in generating streaming media such as HLS or DASH.

![formats](./images/formats.png)

- Containers host video and audio compression codecs.
- HLS: HTTP Live Streaming
- DASH: Dynamic Adaptive Streaming over HTTP

**Lets check out some code**!

Making the Transcode request to convert a mp4 video to a wav audio file:

In [ ]:
url = "https://api.dolby.com/media/transcode"

headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

# Convert the file to a WAV file
body = {
    "inputs": [{ "source" : "https://dolbyio.s3-us-west-1.amazonaws.com/public/shelby/indoors.original.mp4"}],
    "outputs": [
        {
            "id" : "my_mp4",
            "destination" : "dlb://out/airplane-transcoded.wav",
            "kind" : "wav"
        }
    ]
}

response = requests.post(url, json=body, headers=headers)
print(response.json())

**Checking the status:**

In [ ]:
params = {
    "job_id": response.json()["job_id"]
}
stat = requests.get(url, params=params, headers=headers)
print(stat.json())

**Downloading the output:**

In [ ]:
url = "https://api.dolby.com/media/output"
output_path = "airplane-example.wav"
params = {
    "url": "dlb://out/airplane-transcoded.wav",
}

with requests.get(url, params=params, headers=headers, stream=True) as response:
    response.raise_for_status()
    response.raw.decode_content = True
    print("Downloading from {0} into {1}".format(response.url, output_path))
    with open(output_path, "wb") as output_file:
        shutil.copyfileobj(response.raw, output_file)

You should now see `airplane-example.wav` in the side bar!

In [ ]:
IPython.display.Audio("airplane-example.wav")

### Questions?

https://docs.dolby.io/media-apis/docs/transcode-api-guide

## Additional APIs

### Analyze (*Beta*) and Analyze Speech (*Beta*)

- Bigger version of Diagnose

- Breaks out many segments of the audio for quality scores

- Speaker diarization

- Micro level audio defects and insights

**We'll explore Analyze and Analyze Speech in another workshop so stay tuned and let us know if you are interested.**

Explore more here: https://docs.dolby.io/media-apis/docs/analyze-api-guide

### Music Mastering

![mastering](./images/mastering.png)

- Like Enhance but more music specific

- Pick between various music styles

- Professional-sounding audio masters that keep creative intent intact

**If you are interested in Music Mastering reach out to the team!**

Explore more here: https://dolby.io/products/music-mastering/
And check out our Music Mastering partners:

* https://community.soundcloud.com/mastering-on-soundcloud
* https://unitedmasters.zendesk.com/hc/en-us/articles/4404029984019-What-is-Dolby-Mastering-

**Webhooks and Callbacks**

![zapier](./images/zapier.png)

If you prefer not to manually poll for a job to be completed (if processing a large file for example), you can use webhooks and callbacks instead!

To do this, you will need to have a webhook URL with a service that is actively listening for this, such as a Web Service (Express, Django, Rails, etc.), or an integration (Zapier, Slack Apps, etc.). This will trigger a `POST` request, which can be used to automate the next steps of your process.

To read more, see: https://docs.dolby.io/media-apis/docs/webhooks-and-callbacks

### **To Recap...**

- Enhance API
    - Noise reduction
    - Speaker isolation

- Diagnose API
    - Quality scores
    - Learning more about your media

- Transcoding API
    - Media streams
    - Converting to different codecs

- Briefly touched on Analyze and Music Mastering APIs
- Discussed Authentication best practices
- Mentioned Webhooks and Callbacks as automation next steps

If you want to go back and try this again later, here is the GitHub repository: https://github.com/dolbyio-samples/workshop-media-apis-getting-started